In [3]:
import os
import numpy as np
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from scipy import signal
import matplotlib.pyplot as plt
import noisereduce as nr

def create_sample_dataset(speakers=3, utterances=5, duration=3, sr=16000):
    """Simulate creating a dataset by generating synthetic voice-like signals"""
    os.makedirs("speaker_data", exist_ok=True)
    
    for spk in range(speakers):
        os.makedirs(f"speaker_data/speaker_{spk}", exist_ok=True)
        for utt in range(utterances):
            
            
            t = np.linspace(0, duration, int(duration * sr))
            freq = 100 + 50 * spk + np.random.normal(0, 10) 
            mod_freq = 5 + np.random.normal(0, 1) 
            
            carrier = np.sin(2 * np.pi * freq * t)
            modulator = 0.5 * (1 + np.sin(2 * np.pi * mod_freq * t))
            signal = carrier * modulator
            
            signal += 0.8 * np.random.normal(0, 1, len(t))
            
            sf.write(f"speaker_data/speaker_{spk}/utterance_{utt}.wav", signal, sr)

def extract_features(y, sr, n_mfcc=13, n_ceps=13):
    """Extract MFCC and cepstrum features from audio signal"""
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    
    cepstrum = np.fft.irfft(np.log(np.abs(np.fft.rfft(y)) + 1e-10))
    cepstrum = cepstrum[:n_ceps] 
    
    return np.concatenate([mfcc_mean, mfcc_std, cepstrum])

def load_dataset():
    """Load dataset and extract features"""
    X_clean = []
    X_noisy = []
    X_filtered = []
    y = []
    
    for speaker_dir in os.listdir("speaker_data"):
        if not speaker_dir.startswith("speaker_"):
            continue
            
        speaker_id = int(speaker_dir.split("_")[1])
        
        for utterance_file in os.listdir(f"speaker_data/{speaker_dir}"):
            if not utterance_file.endswith(".wav"):
                continue
                
            file_path = f"speaker_data/{speaker_dir}/{utterance_file}"
            y_clean, sr = librosa.load(file_path, sr=None)
            
            features_clean = extract_features(y_clean, sr)
            X_clean.append(features_clean)
            
            y_noisy = y_clean + 0.05 * np.random.normal(0, 1, len(y_clean))
            features_noisy = extract_features(y_noisy, sr)
            X_noisy.append(features_noisy)
            
            y_filtered = signal.lfilter([1, 0.5], [1], y_clean)
            features_filtered = extract_features(y_filtered, sr)
            X_filtered.append(features_filtered)
            
            y.append(speaker_id)
    
    return (np.array(X_clean), np.array(X_noisy), np.array(X_filtered), np.array(y))

def train_and_evaluate(X_train, X_test, y_train, y_test, model_type='knn'):
    """Train and evaluate classifier"""
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    if model_type == 'knn':
        model = KNeighborsClassifier(n_neighbors=3)
    elif model_type == 'svm':
        model = SVC(kernel='rbf', C=10, gamma=0.1)
    else:
        raise ValueError("Invalid model type")
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"\n{model_type.upper()} Results:")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    
    return model, scaler

def main():
    print("Creating sample dataset...")
    create_sample_dataset()
    
    print("Extracting features...")
    X_clean, X_noisy, X_filtered, y = load_dataset()
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y, test_size=0.3, random_state=42)
    
    _, X_test_noisy, _, _ = train_test_split(
        X_noisy, y, test_size=0.3, random_state=42)
    
    _, X_test_filtered, _, _ = train_test_split(
        X_filtered, y, test_size=0.3, random_state=42)
    
    print("\nEvaluating on clean data:")
    knn, scaler = train_and_evaluate(X_train, X_test, y_train, y_test, 'knn')
    svm, _ = train_and_evaluate(X_train, X_test, y_train, y_test, 'svm')
    
    print("\nEvaluating on noisy data:")
    X_test_noisy_scaled = scaler.transform(X_test_noisy)
    y_pred_knn = knn.predict(X_test_noisy_scaled)
    y_pred_svm = svm.predict(X_test_noisy_scaled)
    
    print("\nKNN Results (Noisy):")
    print(classification_report(y_test, y_pred_knn))
    print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")
    
    print("\nSVM Results (Noisy):")
    print(classification_report(y_test, y_pred_svm))
    print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")
    
    print("\nEvaluating on filtered data:")
    X_test_filtered_scaled = scaler.transform(X_test_filtered)
    y_pred_knn = knn.predict(X_test_filtered_scaled)
    y_pred_svm = svm.predict(X_test_filtered_scaled)
    
    print("\nKNN Results (Filtered):")
    print(classification_report(y_test, y_pred_knn))
    print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")
    
    print("\nSVM Results (Filtered):")
    print(classification_report(y_test, y_pred_svm))
    print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

if __name__ == "__main__":
    main()

Creating sample dataset...
Extracting features...

Evaluating on clean data:

KNN Results:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.50      0.67         2
           2       1.00      1.00      1.00         2

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.78         5
weighted avg       0.90      0.80      0.80         5

Accuracy: 0.80

SVM Results:
              precision    recall  f1-score   support

           0       0.20      1.00      0.33         1
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         2

    accuracy                           0.20         5
   macro avg       0.07      0.33      0.11         5
weighted avg       0.04      0.20      0.07         5

Accuracy: 0.20

Evaluating on noisy data:

KNN Results (Noisy):
              precision    recall  f1-score   support


E:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: